# Persistencia con Neon (Postgres)

Este notebook muestra un flujo minimo de LangGraph con persistencia en Neon.
No usa memoria local; todo se guarda en Postgres.


In [2]:
# Instalacion (si hace falta)
%pip install -q langgraph langchain_openai langchain_core langchain_community langgraph-checkpoint-postgres python-dotenv psycopg[binary]


zsh:1: no matches found: psycopg[binary]
Note: you may need to restart the kernel to use updated packages.


In [25]:
# Cargar variables de entorno
import os
from dotenv import load_dotenv

load_dotenv()

database_url = os.environ.get("DATABASE_URL")
if not database_url:
    raise RuntimeError("Define DATABASE_URL en .env para usar Neon/Postgres")


In [4]:
# Importaciones basicas
from typing import TypedDict, List, Dict, Any, Annotated
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langgraph.graph import StateGraph, START, END, add_messages
from langgraph.checkpoint.postgres import PostgresSaver


In [5]:
# Modelo
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1, max_tokens=500)


In [6]:
# Estado personalizado
class EstadoPersonalizado(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    nombre_usuario: str
    intereses: List[str]
    nivel_experiencia: str
    numero_sesion: int
    preguntas_realizadas: int
    preferencias: Dict[str, Any]

def agente_personalizado(state: EstadoPersonalizado) -> EstadoPersonalizado:
    nombre = state.get("nombre_usuario", "Estudiante")
    intereses = state.get("intereses", [])
    nivel = state.get("nivel_experiencia", "principiante")
    sesion = state.get("numero_sesion", 0) + 1
    preguntas = state.get("preguntas_realizadas", 0) + 1

    system_prompt = (
        f"Eres un tutor de IA. Nombre: {nombre}. Nivel: {nivel}. "
        f"Intereses: {intereses if intereses else 'no definidos'}."
    )

    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    respuesta = llm.invoke(messages)

    ultimo_mensaje = state["messages"][-1].content if state["messages"] else ""
    nuevos_intereses = intereses.copy()
    for interes in ["machine learning", "python", "datos"]:
        if interes in ultimo_mensaje.lower() and interes not in nuevos_intereses:
            nuevos_intereses.append(interes)

    return {
        "messages": [respuesta],
        "nombre_usuario": nombre,
        "intereses": nuevos_intereses,
        "nivel_experiencia": nivel,
        "numero_sesion": sesion,
        "preguntas_realizadas": preguntas,
        "preferencias": state.get("preferencias", {}),
    }


In [13]:
# Grafo con persistencia en Neon
builder = StateGraph(EstadoPersonalizado)
builder.add_node("tutor", agente_personalizado)
builder.add_edge(START, "tutor")
builder.add_edge("tutor", END)


telefono = "+56982430581"  # Numero de telefono como identificador unico

with PostgresSaver.from_conn_string(database_url) as checkpointer:
    checkpointer.setup()
    grafo = builder.compile(checkpointer=checkpointer)

    entrada = {"messages": [HumanMessage(content="Hola me llamo Fabian, Me gusta los autos")] }
    config = {"configurable": {"thread_id": telefono}}
    resultado = grafo.invoke(entrada, config=config)
    print(resultado["messages"][-1].content)


¡Hola, Fabián! Es genial que te gusten los autos. Hay tanto que explorar en ese mundo, desde la ingeniería y el diseño hasta la historia de los automóviles y las carreras. ¿Tienes algún tipo de auto favorito o algún aspecto específico que te interese más?


In [14]:
# Grafo con persistencia en Neon
builder = StateGraph(EstadoPersonalizado)
builder.add_node("tutor", agente_personalizado)
builder.add_edge(START, "tutor")
builder.add_edge("tutor", END)


telefono = "+56982430581"  

with PostgresSaver.from_conn_string(database_url) as checkpointer:
    checkpointer.setup()
    grafo = builder.compile(checkpointer=checkpointer)

    entrada = {"messages": [HumanMessage(content="Hola me llamo Fabian, cuales son mis intereses?")] }
    config = {"configurable": {"thread_id": telefono}}
    resultado = grafo.invoke(entrada, config=config)
    print(resultado["messages"][-1].content)


¡Hola, Fabián! Hasta ahora, me has mencionado que te gustan los autos. Si quieres, podemos explorar más sobre ese interés o descubrir otros que puedan gustarte. ¿Te gustaría hablar sobre algún tipo específico de auto, como deportivos, clásicos, eléctricos, o tal vez sobre la mecánica y el mantenimiento? También podríamos hablar de otros temas si te interesa. ¡Dime qué piensas!
